# **Asteroid Prediction**


# Import Python Libraries

In [232]:
#from google.colab import drive
#drive.mount('/content/drive')

In [233]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Reading the Income Classification Dataset

In [234]:
#%cd /content/drive/My Drive/Data

In [235]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [236]:
df = pd.read_csv('/content/drive/MyDrive/Asteroid.csv',  on_bad_lines='skip')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (11,14,15,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [237]:
df.head()

,full_name,a,e,G,i,om,w,q,ad,per_y,...,rot_per,GM,BV,UB,IR,spec_B,spec_T,neo,pha,moid
0,1 Ceres,2.769165,0.076009,0.12,10.594067,80.305532,73.597694,2.558684,2.979647,4.608202,...,9.074170,62.6284,0.713,0.426,NaN,C,G,N,N,1.59478
1,2 Pallas,2.772466,0.230337,0.11,34.836234,173.080063,310.048857,2.133865,3.411067,4.616444,...,7.813200,14.3000,0.635,0.284,NaN,B,B,N,N,1.23324
2,3 Juno,2.669150,0.256942,0.32,12.988919,169.852760,248.138626,1.983332,3.354967,4.360814,...,7.210000,NaN,0.824,0.433,NaN,Sk,S,N,N,1.03454
3,4 Vesta,2.361418,0.088721,0.32,7.141771,103.810804,150.728541,2.151909,2.570926,3.628837,...,5.342128,17.8000,0.782,0.492,NaN,V,V,N,N,1.13948
4,5 Astraea,2.574249,0.191095,NaN,5.366988,141.576604,358.687608,2.082324,3.066174,4.130323,...,16.806000,NaN,0.826,0.411,NaN,S,S,N,N,1.09589


# Column Renaming

In [238]:
readable_df = df.copy()
readable_df = readable_df.rename(columns={"a": "semi-major axis",
                            "e": "eccentricity",
                            "G": "Magnitude slope parameter",
                            "i": "Inclination with ecliptic plane",
                            "om": "Longitude of the ascending node",
                            "w": "argument of perihelion",
                            "q": "perihelion distance(au)",
                            "ad": "aphelion distance(au)",
                            "per_y": "Orbital period",
                            "data_arc": "data arc-span(d)",
                            "condition_code": "Orbit condition code",
                            "n_obs_used": "number of observations used",
                            "H": "Absolute Magnitude parameter",
                            "condition_code": "Orbit condition code",
                            })

In [239]:
readable_df.head()

,full_name,semi-major axis,eccentricity,Magnitude slope parameter,Inclination with ecliptic plane,Longitude of the ascending node,argument of perihelion,perihelion distance(au),aphelion distance(au),Orbital period,...,rot_per,GM,BV,UB,IR,spec_B,spec_T,neo,pha,moid
0,1 Ceres,2.769165,0.076009,0.12,10.594067,80.305532,73.597694,2.558684,2.979647,4.608202,...,9.074170,62.6284,0.713,0.426,NaN,C,G,N,N,1.59478
1,2 Pallas,2.772466,0.230337,0.11,34.836234,173.080063,310.048857,2.133865,3.411067,4.616444,...,7.813200,14.3000,0.635,0.284,NaN,B,B,N,N,1.23324
2,3 Juno,2.669150,0.256942,0.32,12.988919,169.852760,248.138626,1.983332,3.354967,4.360814,...,7.210000,NaN,0.824,0.433,NaN,Sk,S,N,N,1.03454
3,4 Vesta,2.361418,0.088721,0.32,7.141771,103.810804,150.728541,2.151909,2.570926,3.628837,...,5.342128,17.8000,0.782,0.492,NaN,V,V,N,N,1.13948
4,5 Astraea,2.574249,0.191095,NaN,5.366988,141.576604,358.687608,2.082324,3.066174,4.130323,...,16.806000,NaN,0.826,0.411,NaN,S,S,N,N,1.09589


# **Exploration (EDA) & Visualization**

# Handling missing values

In [240]:
readable_df.shape

(839736, 27)

**Drop nan rows (diameter col)**

In [241]:
readable_df = readable_df.dropna(axis=0, subset=['diameter'])

In [242]:
readable_df.shape

(137681, 27)

**Split a dataset into training and testing sets**

In [243]:
dtf_train, dtf_test = train_test_split(readable_df, test_size=0.2)

In [244]:
dtf_train.shape

(110144, 27)

In [245]:
dtf_test.shape

(27537, 27)

**missing values**

In [246]:
dtf_train.isna().sum()

full_name                               0
semi-major axis                         0
eccentricity                            0
Magnitude slope parameter          110050
Inclination with ecliptic plane         0
Longitude of the ascending node         0
argument of perihelion                  0
perihelion distance(au)                 0
aphelion distance(au)                   0
Orbital period                          0
data arc-span(d)                      113
Orbit condition code                    0
number of observations used             0
Absolute Magnitude parameter          613
diameter                                0
extent                             110129
albedo                               1002
rot_per                            101225
GM                                 110131
BV                                 109352
UB                                 109381
IR                                 110143
spec_B                             109048
spec_T                            

**percent missing values**

In [247]:
percent_missing = dtf_train.isnull().sum() * 100 / len(dtf_train)

missing_value = pd.DataFrame({'column': dtf_train.columns,'percent_missing': percent_missing})

missing_value.sort_values('percent_missing', ascending=False)

missing_value

,column,percent_missing
full_name,full_name,0.000000
semi-major axis,semi-major axis,0.000000
eccentricity,eccentricity,0.000000
Magnitude slope parameter,Magnitude slope parameter,99.914657
Inclination with ecliptic plane,Inclination with ecliptic plane,0.000000
Longitude of the ascending node,Longitude of the ascending node,0.000000
argument of perihelion,argument of perihelion,0.000000
perihelion distance(au),perihelion distance(au),0.000000
aphelion distance(au),aphelion distance(au),0.000000
Orbital period,Orbital period,0.000000


**Dropping columns which has more than 90% of missing values**

In [248]:
dropCols = missing_value["column"][missing_value["percent_missing"]>90]

In [249]:
dtf_train.drop(columns = dropCols , inplace = True)
dtf_test.drop(columns = dropCols , inplace = True)

Drop 'full_name' col

In [250]:
dtf_train = dtf_train.drop(['full_name'], axis=1)
dtf_test = dtf_test.drop(['full_name'], axis=1)

**missing values**

In [251]:
dtf_train.isna().sum()

semi-major axis                       0
eccentricity                          0
Inclination with ecliptic plane       0
Longitude of the ascending node       0
argument of perihelion                0
perihelion distance(au)               0
aphelion distance(au)                 0
Orbital period                        0
data arc-span(d)                    113
Orbit condition code                  0
number of observations used           0
Absolute Magnitude parameter        613
diameter                              0
albedo                             1002
neo                                   0
pha                                   0
moid                                  0
dtype: int64

In [252]:
dtf_train.dtypes

semi-major axis                    float64
eccentricity                       float64
Inclination with ecliptic plane    float64
Longitude of the ascending node    float64
argument of perihelion             float64
perihelion distance(au)            float64
aphelion distance(au)              float64
Orbital period                     float64
data arc-span(d)                   float64
Orbit condition code                object
number of observations used          int64
Absolute Magnitude parameter       float64
diameter                            object
albedo                             float64
neo                                 object
pha                                 object
moid                               float64
dtype: object

In [253]:
nan_cols_contiguous = ['data arc-span(d)', 'Absolute Magnitude parameter', 'albedo']

In [254]:
dtf_train = dtf_train.astype({'diameter': 'float64'})
dtf_test= dtf_test.astype({'diameter': 'float64'})

In [255]:
# tf_train = dtf_train.dropna(axis=0, subset=['diameter'])
# tf_test = dtf_test.dropna(axis=0, subset=['diameter'])

In [256]:
dtf_train['data arc-span(d)'].value_counts()

1.0        3436
2.0        1447
7150.0      118
5.0         106
4.0          92
           ... 
17515.0       1
17711.0       1
14090.0       1
3650.0        1
28734.0       1
Name: data arc-span(d), Length: 18124, dtype: int64

In [257]:
#categorical_variables = ['Orbit condition code', 'neo', 'pha']

## Dealing with Categorical variables


Converting all categorical columns into binary columns

In [258]:
categorical_data = dtf_train.select_dtypes(include=['object']).copy()
categorical_data_test = dtf_test.select_dtypes(include=['object']).copy()
categorical_data 

,Orbit condition code,neo,pha
136735,0.0,N,N
621690,9.0,N,N
14856,0,N,N
160495,0.0,N,N
66512,0,N,N
...,...,...,...
248398,0,N,N
147909,0.0,N,N
407322,0,N,N
224482,0,N,N


"One Hot" encoding

In [259]:
#categorical_data = real_diameter_df.select_dtypes(include=['object']).copy()
new_columns = pd.get_dummies(categorical_data, columns=categorical_data.columns, drop_first=True)
new_columns_test = pd.get_dummies(categorical_data_test, columns=categorical_data_test.columns, drop_first=True)

**Define the same columns on train and test**

In [260]:
new_columns, new_columns_test = new_columns.align(new_columns_test, join='left', axis=1)

In [261]:
new_columns = new_columns.fillna(0)
new_columns_test = new_columns_test.fillna(0)

**Drop original categorical cols**

In [262]:
dtf_train = dtf_train.drop(categorical_data.columns, axis = 1)
dtf_test = dtf_test.drop(categorical_data_test.columns, axis = 1)

In [263]:
model_df_train = pd.concat([dtf_train, new_columns], axis=1)
model_df_test = pd.concat([dtf_test, new_columns_test], axis=1)
model_df_train.head()

,semi-major axis,eccentricity,Inclination with ecliptic plane,Longitude of the ascending node,argument of perihelion,perihelion distance(au),aphelion distance(au),Orbital period,data arc-span(d),number of observations used,...,Orbit condition code_2,Orbit condition code_3,Orbit condition code_4,Orbit condition code_5,Orbit condition code_6,Orbit condition code_7,Orbit condition code_8,Orbit condition code_9,neo_Y,pha_Y
136735,3.190421,0.143728,6.841339,69.656155,253.416745,2.731869,3.648973,5.698757,9415.0,297,...,0,0,0,0,0,0,0,0,0,0
621690,2.002942,0.143786,8.778288,265.342100,8.285577,1.714947,2.290937,2.834724,2.0,7,...,0,0,0,0,0,0,0,0,0,0
14856,2.594598,0.283532,16.978259,173.241306,221.557780,1.858945,3.330250,4.179393,10718.0,1183,...,0,0,0,0,0,0,0,0,0,0
160495,3.217366,0.150394,19.166015,167.199938,277.734416,2.733494,3.701237,5.771103,7150.0,317,...,0,0,0,0,0,0,0,0,0,0
66512,2.568345,0.189669,11.698549,156.381892,140.576449,2.081209,3.055480,4.116121,7834.0,986,...,0,0,0,0,0,0,0,0,0,0


In [264]:
new_columns.shape

(110144, 21)

In [265]:
model_df_train.isna().sum()


semi-major axis                       0
eccentricity                          0
Inclination with ecliptic plane       0
Longitude of the ascending node       0
argument of perihelion                0
perihelion distance(au)               0
aphelion distance(au)                 0
Orbital period                        0
data arc-span(d)                    113
number of observations used           0
Absolute Magnitude parameter        613
diameter                              0
albedo                             1002
moid                                  0
Orbit condition code_1                0
Orbit condition code_2.0              0
Orbit condition code_3.0              0
Orbit condition code_4.0              0
Orbit condition code_5.0              0
Orbit condition code_6.0              0
Orbit condition code_7.0              0
Orbit condition code_8.0              0
Orbit condition code_9.0              0
Orbit condition code_0                0
Orbit condition code_1                0


In [266]:
model_df_train.shape

(110144, 35)

In [267]:
model_df_test.shape

(27537, 35)

In [268]:
def predict_missing_values(nan_cols, model_df_train, model_df_test):

  for arg in nan_cols:
    model_copy = model_df_train.copy()
    nan_cols_con_copy = nan_cols.copy()
    nan_cols_con_copy.remove(arg)
    model_copy = model_copy.drop(nan_cols_con_copy, axis=1)
    test_data = model_copy[model_copy[arg].isnull()]
    model_copy.dropna(inplace=True)

    y_train = model_copy[arg]
    X_train = model_copy.drop(arg, axis=1)
    X_test = test_data.drop(arg, axis=1)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    model_df_train.loc[model_df_train[arg].isnull(), arg] = y_pred
    #model_copy_test = model_df_test.copy()

    test_data = model_df_test[model_df_test[arg].isnull()]
    #y_pred_test = model.predict(model_df_test.drop(arg, axis=1))
    y_pred_test = model.predict(test_data.drop(nan_cols, axis=1))
    model_df_test.loc[model_df_test[arg].isnull(), arg] = y_pred_test


In [269]:
predict_missing_values(nan_cols_contiguous, model_df_train, model_df_test)

In [270]:
model_df_test.isna().sum()

semi-major axis                    0
eccentricity                       0
Inclination with ecliptic plane    0
Longitude of the ascending node    0
argument of perihelion             0
perihelion distance(au)            0
aphelion distance(au)              0
Orbital period                     0
data arc-span(d)                   0
number of observations used        0
Absolute Magnitude parameter       0
diameter                           0
albedo                             0
moid                               0
Orbit condition code_1             0
Orbit condition code_2.0           0
Orbit condition code_3.0           0
Orbit condition code_4.0           0
Orbit condition code_5.0           0
Orbit condition code_6.0           0
Orbit condition code_7.0           0
Orbit condition code_8.0           0
Orbit condition code_9.0           0
Orbit condition code_0             0
Orbit condition code_1             0
Orbit condition code_2             0
Orbit condition code_3             0
O

# Asteroids

In [271]:
# plt.figure(figsize=(20,17))
# cor = model_df_train.corr()
# sns.heatmap(model_df_train.corr(method='pearson'),annot=True)
# plt.title("Correlation Between Features")
# plt.show()

In [272]:
model_df_train['diameter'].value_counts()

2.613     39
3.140     39
3.473     38
3.377     38
3.111     37
          ..
13.265     1
13.696     1
9.648      1
29.731     1
7.417      1
Name: diameter, Length: 15287, dtype: int64

## Linear Regression

In [273]:
y_train = model_df_train['diameter']
X_train = model_df_train.drop(['diameter'], axis = 1)
y_test = model_df_test['diameter']
X_test = model_df_test.drop(['diameter'], axis = 1)

In [274]:
X_test.shape

(27537, 34)

In [275]:
X_train.isna().sum().sum()

0

In [276]:
multi_reg = LinearRegression()
multi_reg.fit(X_train , y_train)
pred = multi_reg.predict(X_test)

In [277]:
print('Training Accuracy : ' , multi_reg.score(X_train, y_train))
print('Test Accuracy : ' , multi_reg.score(X_test, y_test))

Training Accuracy :  0.45845271827269973
Test Accuracy :  0.5230363793076928


## Random Forest Regression

In [278]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

In [279]:
print('Training Accuracy : ' , multi_reg.score(X_train, y_train))
print('Test Accuracy : ' , multi_reg.score(X_test, y_test))

Training Accuracy :  0.45845271827269973
Test Accuracy :  0.5230363793076928
